In [2]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np
#import plotly.graph_objects as go
import psycopg2

%matplotlib inline

con = psycopg2.connect (user = 'postgres',
                password = 'postgres',
                port = '5432',
                host = 'localhost',                
                database = 'inicudb')

cur  = con.cursor()

In [6]:
cur1 = con.cursor()
cur1.execute("SELECT DISTINCT(uhid) FROM apollo.baby_detail WHERE dateofadmission >= '2018-07-01' AND dateofadmission <= '2020-05-31' and UHID IN (select distinct(uhid) from apollo.babyfeed_detail UNION select distinct(uhid) from apollo.nursing_intake_output UNION select distinct(uhid) from apollo.baby_prescription UNION select distinct(uhid) from apollo.nursing_vitalparameters UNION select distinct(uhid) from apollo.nursing_bloodgas UNION select distinct(uhid) from apollo.baby_visit UNION select distinct(uhid) from apollo.device_ventilator_detail UNION select distinct(uhid) from apollo.device_monitor_detail UNION select distinct(uhid) from apollo.device_monitor_detail_dump UNION select distinct(uhid) from apollo.device_ventilator_detail_dump UNION select distinct(uhid) from apollo.parent_detail UNION select distinct(uhid) from apollo.birth_to_nicu UNION select distinct(uhid) from apollo.antenatal_history_detail UNION select distinct(uhid) from apollo.sa_resp_rds UNION select distinct(uhid) from apollo.sa_jaundice UNION select distinct(uhid) from apollo.sa_infection_sepsis) and (dischargestatus = 'Death' OR dischargestatus = 'Discharge') and isreadmitted is not true and gestationweekbylmp is not null and birthweight is not null;")
cols1 = list(map(lambda x: x[0], cur1.description))
ds = pd.DataFrame(cur1.fetchall(),columns=cols1)
#ds.sort_values('modificationtime',inplace=True)

In [8]:
cur2 = con.cursor()
cur2.execute("select distinct(b.uhid),l.conception_type, b.gender, b.dateofadmission, b.dischargestatus,b.birthweight,b.weight_galevel,b.weight_centile,b.birthlength,b.birthheadcircumference, b.inout_patient_status, b.gestationweekbylmp, b.gestationdaysbylmp,round( CAST((b.gestationweekbylmp + b.gestationdaysbylmp/7::float) as numeric),2) as Gestation,b.dischargeddate, b.admissionweight, b.baby_type,b.baby_number, b.branchname ,DATE_PART('day',b.dischargeddate - b.dateofadmission) as LOS,c.apgar_onemin, c.apgar_fivemin,c.apgar_tenmin, c.resuscitation,d.isantenatalsteroidgiven, d.mode_of_delivery,z.motherage,e.jaundicestatus as JAUNDICE,  f.eventstatus as SEPSIS,f.progressnotes, g.eventstatus as RDS, g.progressnotes,y.eventstatus as ASPHYXIA,y.progressnotes from apollo.baby_detail as b left join apollo.birth_to_nicu as c on b.uhid = c.uhid and b.episodeid = c.episodeid left join apollo.parent_detail as z on b.uhid = z.uhid and b.episodeid = z.episodeid left join apollo.antenatal_history_detail as d on b.uhid = d.uhid and b.episodeid = d.episodeid left join apollo.sa_jaundice AS e ON b.uhid = e.uhid and e.jaundicestatus = 'Yes' and e.phototherapyvalue='Start' left join apollo.sa_infection_sepsis AS f ON b.uhid = f.uhid and f.eventstatus = 'yes' and f.episode_number = 1 left join apollo.sa_cns_asphyxia AS y ON b.uhid = y.uhid and y.eventstatus = 'yes' and y.episode_number = 1 left join apollo.antenatal_history_detail as l on b.uhid=l.uhid left join apollo.sa_resp_rds AS g ON b.uhid = g.uhid and g.eventstatus = 'Yes' and g.episode_number = 1 and g.uhid IN (select distinct(h.uhid) from apollo.respsupport AS h where h.eventname='Respiratory Distress' and (h.rs_vent_type ='Mechanical Ventilation' OR h.rs_vent_type ='HFO') UNION select distinct(i.uhid) from apollo.sa_resp_rds AS i where i.sufactantname is not null) order by b.dateofadmission")
cols2 = list(map(lambda x: x[0], cur2.description))
dt = pd.DataFrame(cur2.fetchall(),columns=cols2)
#ds.sort_values('modificationtime',inplace=True)

In [10]:
final = pd.DataFrame()
for i in ds.uhid.unique():
    x = dt[dt['uhid']==i]
    final = final.append(x,ignore_index=True)

In [13]:
len(final.uhid.unique())

1574

In [14]:
dh = final.drop_duplicates('uhid',keep='first')

In [16]:
dh.dischargestatus.unique()

array(['Death', 'Discharge'], dtype=object)

In [17]:
dh.to_csv('baseline_20th_june.csv')